In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import json
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from dataclasses import dataclass

In [35]:
driver = webdriver.Chrome()

In [36]:
driver.switch_to.new_window('tab')

In [37]:
tabs = {
 'desafio_02': driver.window_handles[0],
 'desafio_03': driver.window_handles[1],
}

In [38]:
driver.switch_to.window(tabs['desafio_02'])
driver.get('https://curso-web-scraping.pages.dev/#/desafio/2')

In [39]:
driver.switch_to.window(tabs['desafio_03'])
driver.get('https://curso-web-scraping.pages.dev/#/desafio/3')

In [42]:


wait = WebDriverWait(driver, timeout=30, poll_frequency=1)

@dataclass
class User:
    nome: str
    profissao: str
    email: str
    telefone: str
    nome_usuario: str
    estado: str
    link_imagem: str

def get_current_name():
    try:
        driver.switch_to.window(tabs['desafio_03'])
        return driver.find_element(By.CSS_SELECTOR, '#usuario').text 
    except:
        return False

def cadastrar_usuarios(dados_usuarios: list[User]):
    driver.switch_to.window(tabs['desafio_03'])
    for dado_usuario in dados_usuarios:
        input_nome = driver.find_element(By.NAME,"nome")
        input_nome.clear()
        input_nome.send_keys(dado_usuario.nome)
        input_profissao = driver.find_element(By.NAME,"profissao")
        input_profissao.clear()
        input_profissao.send_keys(dado_usuario.profissao)
        input_email = driver.find_element(By.NAME,"email")
        input_email.clear()
        input_email.send_keys(dado_usuario.email)
        input_telefone = driver.find_element(By.NAME,"telefone")
        input_telefone.clear()
        input_telefone.send_keys(dado_usuario.telefone)
        input_usuario = driver.find_element(By.NAME,"usuario")
        input_usuario.clear()
        input_usuario.send_keys(dado_usuario.nome_usuario)
        input_estado = Select(driver.find_element(By.NAME,"estado"))
        input_estado.select_by_visible_text(dado_usuario.estado)
        input_nome.submit()



while get_current_name():
    name = get_current_name()

    driver.switch_to.window(tabs['desafio_02'])
    search_field =  driver.find_element(By.XPATH, '/html/body/div/div/div[2]/main/div[2]/div/div[2]/input')
    search_field.clear()
    search_field.send_keys(name)

    wait.until(EC.element_to_be_clickable(mark=(By.XPATH, '/html/body/div/div/div[2]/main/div[2]/div/div[2]/button')))

    search_button_field =  driver.find_element(By.XPATH, '/html/body/div/div/div[2]/main/div[2]/div/div[2]/button')
    search_button_field.click()

    wait.until(EC.presence_of_all_elements_located(locator=(By.TAG_NAME, 'h3')))

    data_users = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/main/div[2]/div/section/div/div[2]/div')
    dados_usuarios = []
    for data in data_users:
        nome = data.find_element(By.CSS_SELECTOR,'div>h3').text
        profissao = data.find_element(By.CSS_SELECTOR,'div>span').text
        email = data.find_element(By.TAG_NAME,'ul').find_element(By.CSS_SELECTOR,'li:nth-child(1)').text.replace('E-mail: ','')
        telefone = data.find_element(By.TAG_NAME,'ul').find_element(By.CSS_SELECTOR,'li:nth-child(2)').text.replace('Telefone: ','')
        nome_usuario = data.find_element(By.TAG_NAME,'ul').find_element(By.CSS_SELECTOR,'li:nth-child(3)').text.replace('Usuário: ','')
        estado = data.find_element(By.TAG_NAME,'ul').find_element(By.CSS_SELECTOR,'li:nth-child(4)').text.replace('Estado: ','')
        link_imagem = data.find_element(By.TAG_NAME,'img').get_attribute('src')

        dados_usuarios.append(
            User(
                nome,
                profissao,
                email,
                telefone,
                nome_usuario,
                estado,
                link_imagem
            )
        )
    cadastrar_usuarios(dados_usuarios)